In [1]:
import pandas as pd
import util
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import svm
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\radus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
files = util.get_big_files()

df = pd.DataFrame(columns=["doc_text","doc_topic"])
i = 0
for path in files:
    #doc name
    name = path.split("\\")[1][:-4]
    #extract titles
    doc_text = util.extract_title(path) +"\n"+ util.extract_body(path)
    #extract text
    doc_text = doc_text.replace("\n"," ")
    #extract topics
    topics = util.extract_topics(path)
    
    #process topics 
    doc_text = util.process_sentece(doc_text)
    #add topics to dataframe
    df.loc[name] = [doc_text,topics]
    i+=1
    

D:\Labs Master\Text mining - NLP\util.py:28: FutureWarning: This search is broken in 1.3 and earlier, and will be fixed in a future version.  If you rely on the current behaviour, change it to ".//codes[@class='bip:topics:1.0']"
  for element in tree.find("//codes[@class='bip:topics:1.0']"):


In [3]:
result = df.iloc[0] 
print(result)

doc_text     usa top ceo say comput product overcompl compu...
doc_topic                                          [C31, CCAT]
Name: 100034NE, dtype: object


In [4]:
corpus = df['doc_text'].tolist()
util.save_json_data("test.json",{"data": result['doc_text']})

In [5]:
vec = CountVectorizer()
X = vec.fit_transform(corpus)
df_rez = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
df_rez.index = df.index
df_rez['doc_topic'] = df['doc_topic']
# print(df_rez)

d:\labs master\text mining - nlp\venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [6]:
df_rez.to_csv("term_freq_matrix_full.csv",index=True)

In [7]:
#calculate sparsity
sparsity = (df_rez.to_numpy() == 0).mean()
print (f"SPARSITY: {sparsity}")

SPARSITY: 0.9960593682309853


In [8]:
#split the doc in train and test
df_rez['doc_topic'] = MultiLabelBinarizer().fit_transform(df_rez['doc_topic'])
train=df_rez.sample(frac=0.7,random_state=200) #random state is a seed value



test=df_rez.drop(train.index)
# print(train['doc_topic'])
# print(test['doc_topic'])
train_X = train.drop(columns=['doc_topic'])
train_Y = train['doc_topic'].tolist()

test_X = test.drop(columns=['doc_topic'])
test_Y = test['doc_topic'].tolist()

# binarize

# train_X = MultiLabelBinarizer().fit_transform(train_X)
# train_Y = MultiLabelBinarizer().fit_transform(train_Y)
# test_X = MultiLabelBinarizer().fit_transform(test_X)
# test_Y = MultiLabelBinarizer().fit_transform(test_Y)



In [15]:
print(train_X.shape)
# print(train_Y.shape)
print(test_X.shape)
# print(test_Y.shape)
print(train_Y)
print(test_Y)

(4958, 20269)
(2125, 20269)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score,recall_score
from sklearn.metrics import confusion_matrix


model = DecisionTreeClassifier()
model.fit(train_X, train_Y)
pred= model.predict(test_X)
print()
acc = accuracy_score(test_Y, pred)
print(acc)
print(f1_score(test_Y, pred))

In [18]:
from sklearn.metrics import accuracy_score, f1_score, precision_score,recall_score
print(f"PRECISION: {precision_score(test_Y, pred)}")
print(f"RECALL: {recall_score(test_Y, pred)}")

PRECISION: 0.45774647887323944
RECALL: 0.3651685393258427
